In [57]:
import requests
import time
import os, json
import csv
import concurrent.futures
# from functools import partial
import pandas as pd

In [58]:
# url = "https://refract.fosfor.com/magiccodermodel/6188a3e2-bc43-444e-bfaa-8159fca71b3a/score"
url = "http://vllm-serving-service:8888/v1/completions"
# lltoken = "eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJ4WTdTd3k5UE1xaXRDQmNSMm5qcVl6bmoxS3NqZzV3TmdOV0xDVzdyUkhvIn0.eyJleHAiOjE3MzA4ODM3MzgsImlhdCI6MTY5OTI2MTMzOCwiYXV0aF90aW1lIjoxNjk5MjUyMDQ0LCJqdGkiOiJmN2EzMzQwYy1kNDQwLTRlMzUtYjk2ZS04YzBiMTc0Y2RhODAiLCJpc3MiOiJodHRwczovL3JlZnJhY3QtbG9naW4uZm9zZm9yLmNvbS9hdXRoL3JlYWxtcy9tb3NhaWMiLCJhdWQiOlsibW9zYWljLWdhdGVrZWVwZXIiLCJhY2NvdW50Il0sInN1YiI6IjZjMjU4MWU3LWZmMTItNDljNy04MDJmLWI2ZjQzOWQxZDIwMSIsInR5cCI6IkJlYXJlciIsImF6cCI6Im1vc2FpYy1nYXRla2VlcGVyIiwic2Vzc2lvbl9zdGF0ZSI6IjBhY2Y3YWZhLTVmMzMtNGRhZS05OGM3LTQyZDQwYTdlZTM2NiIsImFsbG93ZWQtb3JpZ2lucyI6WyIqIl0sInJlYWxtX2FjY2VzcyI6eyJyb2xlcyI6WyJNTE9QUyIsImxvbmdfbGl2ZWRfdG9rZW4iLCJzcGVjdHJhLWRldmVsb3BlciIsImRlZmF1bHQtcm9sZXMtbW9zYWljIiwicmVmcmFjdC1kZXZlbG9wZXIiLCJvZmZsaW5lX2FjY2VzcyIsImFkbWluIiwidW1hX2F1dGhvcml6YXRpb24iLCJyZWZyYWN0LWFkbWluIl19LCJyZXNvdXJjZV9hY2Nlc3MiOnsiYWNjb3VudCI6eyJyb2xlcyI6WyJtYW5hZ2UtYWNjb3VudCIsIm1hbmFnZS1hY2NvdW50LWxpbmtzIiwidmlldy1wcm9maWxlIl19fSwic2NvcGUiOiJvcGVuaWQgZW1haWwgcHJvZmlsZSIsInNpZCI6IjBhY2Y3YWZhLTVmMzMtNGRhZS05OGM3LTQyZDQwYTdlZTM2NiIsImVtYWlsX3ZlcmlmaWVkIjp0cnVlLCJuYW1lIjoiUmVmcmFjdCBCRlNJIiwicHJlZmVycmVkX3VzZXJuYW1lIjoicmVmcmFjdC5iZnNpQGZvc2Zvci5jb20iLCJnaXZlbl9uYW1lIjoiUmVmcmFjdCIsImZhbWlseV9uYW1lIjoiQkZTSSIsImVtYWlsIjoicmVmcmFjdC5iZnNpQGZvc2Zvci5jb20ifQ.b6SYLgjo9Veo3GmJ8eZjCTNupQjpfMhzsoXdYjWwRtvRnNjBfx0gOqcugO9OcGn-mm8wwpSGI5uiL30-I6SdWBjsf1ur6GztoX7j-nP_3SrJJn3UhNNqIO8LbsPi5gGRTzWtnfjz92BF1YaCXxQwPY0P_aa8vJ6JxZz5Uctn9aIPIJZZnnjC_GPXtXurmshM_tEN2kwCjhEyr7wYzRqUoMtBGfpLjZREBzgZY-x6JyYiXNtycb1d6PFcCXf7nJVV8ienEC_x7OuciDzfeqd-SQnImvAHH7rqFdi9smBN08AbkDS2uAbMrokHrmbiBpaimrR013VwCWz2KL5QYlWleA"
request_payload_file = '/data/Magicoder_payloads.txt'
report_path = '/data/vLLM_5Concurrency_1gpu.csv'
df = pd.DataFrame(columns=['Response', 'Response_status', 'Time (seconds)'])

In [59]:
payload = {"prompt": "Problem: Write a function in python to check if two strings are anagrams.Input: Strings s1 and s2.Output: Boolean, indicating whether s1 and s2 are anagrams.", 
           "max_tokens": 200,
           "temperature": 0,
           "model": "/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66"}

In [60]:
def read_payloads(file_path):
    with open(file_path, 'r') as file:
        for line in file:
            yield line.strip()

In [61]:
def my_decorator(func):
    def calculate_execution_time(*args, **kwargs):
        global df
        start_time = time.time()
        result= func(*args, **kwargs)
        end_time = time.time()
        execution_time = end_time - start_time
        data = {'Response':result.text, 'Response_status':result.status_code, 'Time (seconds)':f'{execution_time:.6f}'}
        df = df.append(data, ignore_index=True)
        print(f"Execution time: {execution_time:.6f} seconds \n\n")
        return result
    return calculate_execution_time

In [62]:
def save_report(file_path, df):
    print(df)
    file_exists = os.path.exists(file_path)
    df.to_csv(file_path, index=False)

In [63]:
@my_decorator
def inference(payload_data=None):
    headers = {
        'Content-Type': 'application/json'
    }
    try:
        response = requests.request("POST", url, headers=headers, json=payload_data)
        print(f"Prompt==> {payload_data['prompt']} \n {response.text}")
        return response
    except Exception as e:
        print(e)
        

In [64]:
#Model Inference(Request)
if __name__ == "__main__":
    # Using the generator function to read lines 
    prompts = read_payloads(request_payload_file)
    
    global df
    for i in range(1):
        with concurrent.futures.ThreadPoolExecutor(max_workers=1) as executor:
            futures = [executor.submit(inference, {**payload, 'prompt': prompt}) for prompt in prompts]
            concurrent.futures.wait(futures)
    save_report(report_path, df)

Prompt==> {"payload":"Problem: Write a function in python to reverse a given string.Input: String s.Output: String, the reversed version of s."} 
 {"id":"cmpl-a702e04927ea4e67b5737c2eb2c22bc5","object":"text_completion","created":434491,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\ndef reverse_string(s):\n    return s[::-1]\n\ns = input(\"Enter a string: \")\nprint(\"Reversed string is: \", reverse_string(s))","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":36,"total_tokens":83,"completion_tokens":47}}
{'Response': '{"id":"cmpl-a702e04927ea4e67b5737c2eb2c22bc5","object":"text_completion","created":434491,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\\n\\n\\ndef reverse_string(s):\\n    return s[::-1]\\n\\ns = input(\\"Enter a string: \\")\\nprint(\\"Reversed string is: \\", re

Prompt==> {"payload":"Problem: Write a function in java to calculate the sum of the digits of a given positive integer.Input: Positive integer num.Output: Integer, the sum of the digits of num."} 
 {"id":"cmpl-f7d46f42b0d842cf942abbddbdf2d7bf","object":"text_completion","created":434494,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":48,"total_tokens":49,"completion_tokens":1}}
{'Response': '{"id":"cmpl-f7d46f42b0d842cf942abbddbdf2d7bf","object":"text_completion","created":434494,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":48,"total_tokens":49,"completion_tokens":1}}', 'Response_status': 200, 'Time (seconds)': '0.114699'}
Execution time: 0.114699 seconds 


          

Prompt==> {"payload":"Problem: Write a function in python to check if a given string is a palindrome.Input: String s.Output: Boolean, indicating whether s is a palindrome."} 
 {"id":"cmpl-27a46052bc4e40a58db59476af11f4bc","object":"text_completion","created":434491,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\ndef is_palindrome(s):\n    return s == s[::-1]\n\nprint(is_palindrome('madam'))  # True\nprint(is_palindrome('hello'))  # False","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":44,"total_tokens":98,"completion_tokens":54}}
{'Response': '{"id":"cmpl-27a46052bc4e40a58db59476af11f4bc","object":"text_completion","created":434491,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\\n\\ndef is_palindrome(s):\\n    return s == s[::-1]\\n\\nprint(is_palindrome(\'madam\'))  # True\\nprin

Prompt==> {"payload":"Problem: Write a function in python to transpose a given matrix.Input: 2D list representing a matrix.Output: 2D list, the transpose of the input matrix."} 
 {"id":"cmpl-7884ad610f6e46259b0282c1c33a9053","object":"text_completion","created":434491,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\ndef transpose(matrix):\n    return [[matrix[j][i] for j in range(len(matrix))] for i in range(len(matrix[0]))]\n\n# Test the function\nmatrix = [[1, 2, 3], [4, 5, 6], [7, 8, 9]]\nprint(transpose(matrix))","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":46,"total_tokens":132,"completion_tokens":86}}
{'Response': '{"id":"cmpl-7884ad610f6e46259b0282c1c33a9053","object":"text_completion","created":434491,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\\n\\n\\ndef transpose(

Prompt==> {"payload":"Problem: Write a function in python to calculate the sum of the digits of a given positive integer.Input: Positive integer num.Output: Integer, the sum of the digits of num."} 
 {"id":"cmpl-5f9ca38820f54f0da5b65083a031a5d2","object":"text_completion","created":434491,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\ndef sum_of_digits(num):\n    return sum(int(digit) for digit in str(num))\n\n# Test the function\nprint(sum_of_digits(1234))  # Output: 10\nprint(sum_of_digits(5678))  # Output: 26\nprint(sum_of_digits(9012))  # Output: 10","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":48,"total_tokens":151,"completion_tokens":103}}
{'Response': '{"id":"cmpl-5f9ca38820f54f0da5b65083a031a5d2","object":"text_completion","created":434491,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{

...

Prompt==> {"payload":"Problem: Build a function in jQuery to toggle between light and dark modes for a website.Input: None.Output: None (update the website theme)."} 
 {"id":"cmpl-ad417e87b1d344999f4a005c712d12fa","object":"text_completion","created":434511,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":38,"total_tokens":39,"completion_tokens":1}}
{'Response': '{"id":"cmpl-ad417e87b1d344999f4a005c712d12fa","object":"text_completion","created":434511,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":38,"total_tokens":39,"completion_tokens":1}}', 'Response_status': 200, 'Time (seconds)': '0.115691'}
Execution time: 0.115691 seconds 


                                        

Prompt==> {"payload":"Problem: Build a function in jQuery to populate an HTML table with data from an array of objects.Input: Array of objects, each representing a row in the table.Output: None (update the HTML table)."} 
 {"id":"cmpl-b172aac3f2154cd18df8711dba5897a3","object":"text_completion","created":434511,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":51,"total_tokens":52,"completion_tokens":1}}
{'Response': '{"id":"cmpl-b172aac3f2154cd18df8711dba5897a3","object":"text_completion","created":434511,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":51,"total_tokens":52,"completion_tokens":1}}', 'Response_status': 200, 'Time (seconds)': '0.116567'}
Execution time: 0.116

Prompt==> {"payload":"Problem: Write a function in c++ to transpose a given matrix.Input: 2D list representing a matrix.Output: 2D list, the transpose of the input matrix."} 
 {"id":"cmpl-19da37a5d43a4568ae33a592eb11297f","object":"text_completion","created":434498,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\nHere is a simple C++ function to transpose a given matrix:\n\n```cpp\n#include <iostream>\n#include <vector>\n\nstd::vector<std::vector<int>> transpose(std::vector<std::vector<int>>& matrix) {\n    int rows = matrix.size();\n    int cols = matrix[0].size();\n\n    std::vector<std::vector<int>> transposed(cols, std::vector<int>(rows));\n\n    for (int i = 0; i < rows; i++) {\n        for (int j = 0; j < cols; j++) {\n            transposed[j][i] = matrix[i][j];\n        }\n    }\n\n    return transposed;\n}\n\nint main() {\n    std::vector<std::vector<int>> matrix =","logprobs":null,"

Prompt==> {"payload":"Problem: Write a function in Ruby to calculate the sum of the digits of a given positive integer.Input: Positive integer num.Output: Integer, the sum of the digits of num."} 
 {"id":"cmpl-e76dcb23461c44bfa3dfbcaad6833aaa","object":"text_completion","created":434506,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\ndef sum_of_digits(num)\n  num.to_s.chars.map(&:to_i).sum\nend\n\nputs sum_of_digits(1234) # Output: 10\nputs sum_of_digits(5678) # Output: 26","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":49,"total_tokens":127,"completion_tokens":78}}
{'Response': '{"id":"cmpl-e76dcb23461c44bfa3dfbcaad6833aaa","object":"text_completion","created":434506,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\\n\\ndef sum_of_digits(num)\\n  num.to_s.chars.map(&:to_i).sum\\nen

Prompt==> {"payload":"Problem: Write a function in Ruby to perform binary search on a sorted list.Input: Sorted list arr and target value target.Output: Integer, the index of target in arr (or -1 if not found)."} 
 {"id":"cmpl-49160a94d3fd44628b7a38b5d9cbbb07","object":"text_completion","created":434508,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\ndef binary_search(arr, target)\n  low = 0\n  high = arr.length - 1\n\n  while low <= high\n    mid = (low + high) / 2\n\n    if arr[mid] == target\n      return mid\n    elsif arr[mid] < target\n      low = mid + 1\n    else\n      high = mid - 1\n    end\n  end\n\n  return -1\nend\n\narr = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]\ntarget = 5\nputs binary_search(arr, target)  # Output: 4\n\ntarget = 11\nputs binary_search(arr, target)  # Output: -1","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":53,"total_tokens":235,"completion_tokens

In [ ]:
# import aiohttp
# import asyncio
# import time

# start_time = time.time()
# headers={"Content-type":"application/json"} 
# async def main():

#     async with aiohttp.ClientSession(trust_env = True, headers=headers) as session:

# #         for payload in payloads:
# #             url='https://refract.fosfor.com/openchatfinetunedimqlgenerator/d0721c53-c9f5-4d79-a988-2d21df46198f/score' 
#         async with session.post(url, json=payload) as resp:
#                 mql = await resp.json(content_type=None)
#                 print(mql)

# await main()
# print("--- %s seconds ---" % (time.time() - start_time))